In [ ]:
from azure.storage.filedatalake import DataLakeServiceClient

In [2]:
ano_mes = 202503
# Define URL e caminho local do ZIP
url = f"https://cnes.datasus.gov.br/EstatisticasServlet?path=BASE_DE_DADOS_CNES_{ano_mes}.ZIP"
zip_path = f"/Users/caio.maximiano/pessoal/cnes-project-analysis/local_storage/zip/BASE_DE_DADOS_CNES_{ano_mes}.ZIP"

# Faz download com wget
import os
print(f"Iniciando download para o período {ano_mes}")
os.system(f"wget -O {zip_path} {url}")

Iniciando download para o período 202503


--2025-07-12 12:47:16--  https://cnes.datasus.gov.br/EstatisticasServlet?path=BASE_DE_DADOS_CNES_202503.ZIP
Resolving cnes.datasus.gov.br (cnes.datasus.gov.br)... 189.28.139.195
Connecting to cnes.datasus.gov.br (cnes.datasus.gov.br)|189.28.139.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/Users/caio.maximiano/pessoal/cnes-project-analysis/local_storage/zip/BASE_DE_DADOS_CNES_202503.ZIP’

     0K .......... .......... .......... .......... ..........  676K
    50K .......... .......... .......... .......... ..........  286K
   100K .......... .......... .......... .......... .......... 13.3M
   150K .......... .......... .......... .......... .......... 1.80M
   200K .......... .......... .......... .......... .......... 1019K
   250K .......... .......... .......... .......... .......... 1.37M
   300K .......... .......... .......... .......... ..........  806K
   350K .......... .......... .......... .........

0

In [ ]:
from azure.storage.filedatalake import DataLakeFileClient
from tqdm import tqdm
import os
import time

def upload_to_adls(
    file_client: DataLakeFileClient,
    local_file_path: str,
    chunk_size: int = 4 * 1024 * 1024,  # 4MB
    max_retries: int = 3
):
    """
    Faz upload de um arquivo local para ADLS Gen2 usando append/flush com barra de progresso e retry.
    """
    file_size = os.path.getsize(local_file_path)
    offset = 0

    try:
        file_client.create_file()
    except Exception as e:
        print(f"⚠️ Aviso: arquivo já pode existir: {e}")

    with open(local_file_path, "rb") as f, tqdm(total=file_size, unit='B', unit_scale=True, desc=os.path.basename(local_file_path)) as pbar:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break

            # Retry logic
            for attempt in range(max_retries):
                try:
                    file_client.append_data(data=chunk, offset=offset, length=len(chunk))
                    break
                except Exception as e:
                    wait_time = 2 ** attempt
                    print(f"⚠️ Tentativa {attempt + 1} falhou. Esperando {wait_time}s: {e}")
                    time.sleep(wait_time)
                    if attempt == max_retries - 1:
                        raise e  # Última tentativa falhou

            offset += len(chunk)
            pbar.update(len(chunk))

    file_client.flush_data(offset)
    print(f"✅ Upload finalizado para: {file_client.path_name}")


In [7]:
import zipfile
import os
from azure.storage.filedatalake import DataLakeServiceClient

# Configurações do Azure
account_name = "caiostorageaccountdev"
account_key = "IExmwuvHY7HX2rYSouaQAhAEjFrXYMfGpKIKFFMAVlz8B5rOn8/wQX8Np7yXGQSgneFel74yDwq7+AStvLQjJQ=="
file_system_name = "bronze"
# extract_dir = f"/tmp/cnes_extract_{ano_mes}"
extract_dir = f"/Users/caio.maximiano/pessoal/cnes-project-analysis/local_storage/csv/cnes_extract_{ano_mes}"
target_dir = f"cnes/{ano_mes}"  # pasta no container

# Inicializa cliente do Data Lake
def get_datalake_client():
    return DataLakeServiceClient(
        account_url=f"https://{account_name}.dfs.core.windows.net",
        credential=account_key
    )

# Descompacta o ZIP
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Envia os CSVs para o container "silver"
dl_service = get_datalake_client()
file_system_client = dl_service.get_file_system_client(file_system=file_system_name)

for root, _, files in os.walk(extract_dir):
    for file_name in files:
        if file_name.lower().endswith(".csv"):
            local_file_path = os.path.join(root, file_name)
            remote_path = f"{target_dir}/{file_name}"

            print(f"Enviando {file_name} para abfss://{file_system_name}@{account_name}.dfs.core.windows.net/{remote_path}")

            file_client = file_system_client.get_file_client(remote_path)
            with open(local_file_path, "rb") as f:
                file_client.upload_data(f, overwrite=True)


print("Todos os CSVs foram enviados com sucesso para o container 'silver'.")


Enviando tbInstituicaoAvaliadora202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/tbInstituicaoAvaliadora202503.csv
Enviando tbIncentivos202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/tbIncentivos202503.csv
Enviando tbSubTpModVinculo202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/tbSubTpModVinculo202503.csv
Enviando tbDsei202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/tbDsei202503.csv
Enviando tbTipoAbrangencia202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/tbTipoAbrangencia202503.csv
Enviando rlEstabEndCompl202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/rlEstabEndCompl202503.csv
Enviando rlEquipeAldeia202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/rlEquipeAldeia202503.csv
Enviando rlEstabSamu202503.csv para abfss://bronze

ServiceResponseError: ('Connection aborted.', TimeoutError('The write operation timed out'))

In [9]:
from azure.storage.blob import BlobServiceClient
import os

account_name = "caiostorageaccountdev"
account_key = "IExmwuvHY7HX2rYSouaQAhAEjFrXYMfGpKIKFFMAVlz8B5rOn8/wQX8Np7yXGQSgneFel74yDwq7+AStvLQjJQ=="
file_system_name = "bronze"  # container name
ano_mes = "202503"

local_folder = f"/Users/caio.maximiano/pessoal/cnes-project-analysis/local_storage/csv/cnes_extract_{ano_mes}"
target_dir = f"cnes/{ano_mes}"

blob_service_client = BlobServiceClient(
    account_url=f"https://{account_name}.blob.core.windows.net",
    credential=account_key
)

container_client = blob_service_client.get_container_client(file_system_name)

for root, _, files in os.walk(local_folder):
    for file_name in files:
        if file_name.lower().endswith(".csv"):
            local_path = os.path.join(root, file_name)
            blob_path = f"{target_dir}/{file_name}"

            print(f"Enviando {file_name} para abfss://{file_system_name}@{account_name}.dfs.core.windows.net/{blob_path}")

            with open(local_path, "rb") as data:
                container_client.upload_blob(
                    name=blob_path,
                    data=data,
                    length=os.path.getsize(local_path),
                    overwrite=True
                )

print("Upload finalizado com BlobClient.")


Enviando tbInstituicaoAvaliadora202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/tbInstituicaoAvaliadora202503.csv
Enviando tbIncentivos202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/tbIncentivos202503.csv
Enviando tbSubTpModVinculo202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/tbSubTpModVinculo202503.csv
Enviando tbDsei202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/tbDsei202503.csv
Enviando tbTipoAbrangencia202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/tbTipoAbrangencia202503.csv
Enviando rlEstabEndCompl202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/rlEstabEndCompl202503.csv
Enviando rlEquipeAldeia202503.csv para abfss://bronze@caiostorageaccountdev.dfs.core.windows.net/cnes/202503/rlEquipeAldeia202503.csv
Enviando rlEstabSamu202503.csv para abfss://bronze

ResourceExistsError: Blob operation is not supported.
RequestId:93659eb9-901e-0000-5e4a-f371dd000000
Time:2025-07-12T16:34:45.7291059Z
ErrorCode:BlobOperationNotSupported
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobOperationNotSupported</Code><Message>Blob operation is not supported.
RequestId:93659eb9-901e-0000-5e4a-f371dd000000
Time:2025-07-12T16:34:45.7291059Z</Message></Error>